In [44]:
import json
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import rank_bm25 as bm25
import pandas as pd
import numpy as np
import re
import string
from gensim.parsing.preprocessing import remove_stopwords

In [22]:
wd = '/media/sf_year2/DD2430-Project/Podcast-Segment-Retrieval-Spotify/'
os.chdir(wd)
files = os.listdir('./data/training_episodes')
files = [f for f in files if f != 'place_holder']
#files

In [10]:
with open(os.path.join('./data', 'training_sub.json'), 'rb') as f:
    training_data = json.load(f)

training_docs = training_data.keys()


In [68]:
def strip_punctuation(text):
    return text.translate(text.maketrans("", "", string.punctuation))


def clean_text(text):
    t = text.lower()
    t = strip_punctuation(t)
    t = remove_stopwords(t)
    return t
    
# Simon's  extraction function
def extract_segments(path):
    """Given path to json file containing an episode extracts all segments of that episode, 
    including start and end time of each segment."""
    with open(path, "r") as read_file:
        episode = json.load(read_file)
    segments=[]
    segment_transcripts = []
    #had to do "manual" iteration due to irregularities in data
    iter=0
    for segment in episode["results"]:
        seg_result={}
        
        #make sure there is only one dict in this list (should be true according to dataset description)
        assert len(segment["alternatives"])==1
        segment_dict=segment["alternatives"][0]
        #sometimes "alternatives" dict is empty...
        if "words" and "transcript"  in segment_dict:
            #add segment number
            seg_result["segNum"]=iter
            #add timestamp of the first word in this segment
            seg_result["startTime"]=segment_dict["words"][0]["startTime"]
            #add timestamp of the last word in this segment
            seg_result["endTime"]=segment_dict["words"][-1]["endTime"]
            #add transcript of this segment 
            tr = segment_dict["transcript"]
            seg_result["transcript"]=tr
            segment_transcripts.append(tr)
            segments.append(seg_result)
            iter+=1
            
    return {'segments': segments, "full_text": "".join(segment_transcripts)}


In [69]:
training_sub = {}

for filename in files:
#     with open(os.path.join("./data/training_episodes", filename), 'rb') as f:
#         dat = json.load(f)
    
    training_sub[os.path.splitext(filename)[0]] = extract_segments(os.path.join("./data/training_episodes", filename))

# training_sub

In [71]:
t = training_sub[list(training_sub.keys())[0]]['full_text']
# print(t)
clean_text(t)

'hi im tanya thompson creator executive producer nightlight horror podcast featuring creepy tales black writers world week special treat want thank newest patrons sarah brianna merit jonathan byron tish kanani hope said right im sorry didnt thank patrons helping support podcast podcast patrons join nightlight legion patreon going patreoncom slash nightlight pot onetime donation paypal dot slash nightlight podcast weeks story author ebony dunbar percivals universe month aired bennett benefit week airing london calling story meet werewolves discover actually happened alexander percival sid turn lights enjoy percivals london calling ebony dunbar damn damn damn elena cursed ran alleyway darling language anna maria said taking tight left turn lifting voluminous skirt trip eleanor followed close making turn sending circle going fast truth maria hear creature catching turned right waiting arms vampire damn anna maria came stop eleanor smirk moved stand anna marias rolling shoulders ready figh

In [12]:
all_segments = []
all_starts = []
all_ends = []
corpus = []
for filename in files:
    with open(os.path.join('./data', filename), 'rb') as f:
        dat = json.load(f)

    segments = []
    starts = []
    ends = []
    for alt in dat['results']:
        if 'transcript' in alt['alternatives'][0].keys():
            seg = alt['alternatives'][0]['transcript']
            segments.append(seg)
            s = alt['alternatives'][0]['words'][0]['startTime']
            starts.append(s)
            e = alt['alternatives'][0]['words'][-1]['endTime']
            ends.append(e)

    text = "".join(segments)
    all_segments.append(segments)
    all_starts.append(starts)
    all_ends.append(ends)
    corpus.append(text)

In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.9, stop_words={'english'})
X = vectorizer.fit_transform(corpus)
w = 1998
print(vectorizer.get_feature_names()[w])
print(X[0, w])
print(X[1, w])

christmas
0.0
0.012982360840821777


In [22]:
flatten = lambda l: [item for sublist in l for item in sublist]
flat_segments = flatten(all_segments)
tokenized_segments = [x.split(" ") for x in flat_segments]
# tokenized_segments

In [25]:
bm25 = BM25Okapi(tokenized_segments)

query = "climate change strike"
tokenized_query = query.split(" ")

bm25.get_top_n(tokenized_query, flat_segments, n=2)

[" I don't like that strike the fuck up. What is Donkey Kong, you know bang, you know, when you hit them off of over the head, I'll give you got a Donkey Kong. Laughs well, oh my Heavens, I'm almost gonna beat the Rams your will, you know, when they're you had to lose feeling in her eye. That was almost me. Uh, see, you know, I feel like if you go down hey,",
 " Well rounded Queen shout-out and well Rodney King's as well. Yes, baby. I'm telling you call switching like a mug sometimes better than what's up. What you do got to work voice not because I don't got it on the phone like that and you go but when I did I get on the phone you had to work boys. I changed my posture and if you change your posture, yeah because you got to get some air right in here."]